# preprocessing

In [6]:
!git clone https://github.com/ItamarBerger/TabularDS

fatal: destination path 'TabularDS' already exists and is not an empty directory.


In [7]:
# load dataset
import pandas as pd
retails = pd.read_csv('/content/TabularDS/final_project/code/data/online_retail.csv')
data = retails.copy()

In [8]:
retails.head()
# print(len(retails))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [9]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data.dropna(subset=['InvoiceNo', 'Description'], inplace=True)

transactions = data.groupby('InvoiceNo')['Description'].apply(list).tolist()

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Convert the transaction data into a one-hot encoded DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori to find frequent itemsets (min_support can be adjusted based on your needs)
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)

# Generate association rules (min_threshold can be adjusted for confidence)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Display the results
print(rules)


In [10]:
# transforming raw text into IDs of items
retails['DescriptionID'] = retails['Description'].astype('category').cat.codes

# aggregating each transaction as set of items for each Invoice
retails = retails.groupby('InvoiceNo')['DescriptionID'].apply(set).reset_index()
# retails_with_ids.head()

In [11]:
retails.head()

,InvoiceNo,DescriptionID
0,536365,"{3918, 913, 3926, 1910, 3158, 1529, 2911}"
1,536366,"{1650, 1653}"
2,536367,"{1290, 1803, 568, 2814, 1746, 2034, 244, 2743,..."
3,536368,"{1833, 454, 4062, 2833}"
4,536369,{319}


In [12]:
from mlxtend.preprocessing import TransactionEncoder

# Convert sets to lists
transactions = retails["DescriptionID"].apply(list).tolist()  # Convert sets to lists

# Apply TransactionEncoder
encoder = TransactionEncoder()
encoded_array = encoder.fit(transactions).transform(transactions)

# Convert to a DataFrame
df_encoded = pd.DataFrame(encoded_array, columns=encoder.columns_)



In [13]:
print(df_encoded.head())  # Check transformed DataFrame


    0      1      2      3      4      5      6      7      8      9     ...  \
0  False  False  False  False  False  False  False  False  False  False  ...   
1  False  False  False  False  False  False  False  False  False  False  ...   
2  False  False  False  False  False  False  False  False  False  False  ...   
3  False  False  False  False  False  False  False  False  False  False  ...   
4  False  False  False  False  False  False  False  False  False  False  ...   

    4213   4214   4215   4216   4217   4218   4219   4220   4221   4222  
0  False  False  False  False  False  False  False  False  False  False  
1  False  False  False  False  False  False  False  False  False  False  
2  False  False  False  False  False  False  False  False  False  False  
3  False  False  False  False  False  False  False  False  False  False  
4  False  False  False  False  False  False  False  False  False  False  

[5 rows x 4223 columns]


#Apriori for rule mining

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules

# Convert transactions to a DataFrame
itemset = set(item for transaction in retails for item in transaction)
data_dict = [{item: (item in transaction) for item in itemset} for transaction in retails]
df = pd.DataFrame(data_dict)

# Apply Apriori algorithm
min_support = 0.5  # Minimum support threshold
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)



In [15]:
# Generate association rules
min_confidence = 0.03  # Minimum confidence threshold
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)

print("Frequent Itemsets:")
print(frequent_itemsets)
print("\nAssociation Rules:")
print(rules)

Frequent Itemsets:
      support                           itemsets
0         0.5                                (t)
1         1.0                                (i)
2         1.0                                (o)
3         1.0                                (e)
4         0.5                                (N)
...       ...                                ...
2234      0.5     (t, i, o, I, c, s, n, r, p, D)
2235      0.5     (t, i, e, I, c, s, n, r, p, D)
2236      0.5     (t, o, e, I, c, s, n, r, p, D)
2237      0.5     (i, o, e, I, c, s, n, r, p, D)
2238      0.5  (t, i, o, e, I, c, s, n, r, p, D)

[2239 rows x 2 columns]

Association Rules:
       antecedents                     consequents  antecedent support  \
0              (t)                             (i)                 0.5   
1              (i)                             (t)                 1.0   
2              (t)                             (o)                 0.5   
3              (o)                             (t)  

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


In [16]:
# Filter interesting rules
interesting_rules = rules[(rules['confidence'] >= 0.8) & (rules['lift'] > 1.2) & (rules['support'])]
print("\nMost Interesting Association Rules:")
print(interesting_rules.head(100))



Most Interesting Association Rules:
    antecedents consequents  antecedent support  consequent support  support  \
8           (t)         (D)                 0.5                 0.5      0.5   
9           (D)         (t)                 0.5                 0.5      0.5   
10          (t)         (s)                 0.5                 0.5      0.5   
11          (s)         (t)                 0.5                 0.5      0.5   
14          (t)         (r)                 0.5                 0.5      0.5   
..          ...         ...                 ...                 ...      ...   
327         (D)      (t, n)                 0.5                 0.5      0.5   
328      (t, r)         (D)                 0.5                 0.5      0.5   
329      (t, D)         (r)                 0.5                 0.5      0.5   
330      (r, D)         (t)                 0.5                 0.5      0.5   
331         (t)      (r, D)                 0.5                 0.5      0.5   

  

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

item_sim_matrix = cosine_similarity(df_onehot.T)  # Item-item similarity
item_sim_df = pd.DataFrame(item_sim_matrix, index=df_onehot.columns, columns=df_onehot.columns)


NameError: name 'df_onehot' is not defined

In [ ]:
# perform item-based CF for rule mining
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_item_ratings(invoice_items, item_sim_df):
    """
    Calculate predicted rating for items not in the invoice.
    Rating(Item X) = sum(sim(Item X, item_in_invoice)) / sum(similarities)
    """
    scores = {}
    norm_factor = 0

    for item in invoice_items:
        similar_items = item_sim_df[item].drop(index=invoice_items, errors="ignore")  # Remove existing items
        for sim_item, similarity in similar_items.items():
            scores[sim_item] = scores.get(sim_item, 0) + similarity  # Aggregate similarity scores
            norm_factor += similarity  # Normalize by sum of similarities

    if norm_factor == 0:
        return {}

    # Normalize scores
    for item in scores:
        scores[item] /= norm_factor

    return scores

In [ ]:
from tqdm.notebook import tqdm
threshold = 0.2  # Min rating for an item to be considered

rules = []
for invoice in tqdm(df_onehot.index,desc="Processing Raiting"):
    existing_items = df_onehot.columns[df_onehot.loc[invoice] == 1].tolist()
    predicted_ratings = compute_item_ratings(existing_items, item_sim_df)

    # Select items with rating above threshold
    recommended_items = {item for item, score in predicted_ratings.items() if score > threshold}

    if recommended_items:
        rules.append((set(existing_items), recommended_items))  # Format: {A, B, C} → {D, F}

rules_df = pd.DataFrame(rules, columns=["Antecedent", "Consequent"])


In [ ]:
# compute Confidence and Lift
def compute_rule_metrics(rules_df, df_onehot):
    total_invoices = len(df_onehot)
    metrics = []

    for _, row in rules_df.iterrows():
        antecedent = row["Antecedent"]
        consequent = row["Consequent"]

        support_A = (df_onehot[list(antecedent)].sum(axis=1) == len(antecedent)).mean()
        support_B = (df_onehot[list(consequent)].sum(axis=1) == len(consequent)).mean()
        support_A_B = ((df_onehot[list(antecedent)].sum(axis=1) == len(antecedent)) &
                       (df_onehot[list(consequent)].sum(axis=1) == len(consequent))).mean()

        confidence = support_A_B / support_A if support_A > 0 else 0
        lift = confidence / support_B if support_B > 0 else 0
        interestingness = confidence * lift  # Tradeoff metric

        metrics.append((antecedent, consequent, confidence, lift, interestingness))

    return pd.DataFrame(metrics, columns=["Antecedent", "Consequent", "Confidence", "Lift", "Interestingness"])

rules_eval_df = compute_rule_metrics(rules_df, df_onehot)




In [ ]:
# Select Top 100 Most Interesting Rules
top_100_rules = rules_eval_df.sort_values(by="Interestingness", ascending=False).head(100)
print(top_100_rules)